In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV ,KFold
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.calibration import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
df = pd.read_csv('/Users/hneen./iCloud Drive (Archive)/Desktop/L9/Graduation project/colon_predicitions/Data/colon-dataset-processed.csv')
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])
df

,Class,Age,p16540_C/C,p16540_G/C,p16540_G/G,p16580_C/C,p16580_C/T,p16580_T/T,mdm2_G/G,mdm2_G/T,mdm2_T/T,GAL3_A/A,GAL3_C/A,GAL3_C/C,TIM1_C/C,TIM1_G/C,TIM1_G/G
0,1,49,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False
1,1,49,False,True,False,True,False,False,True,False,False,True,False,False,False,True,False
2,1,49,False,True,False,True,False,False,False,True,False,False,True,False,True,False,False
3,1,36,False,True,False,True,False,False,False,False,True,False,True,False,True,False,False
4,1,49,True,False,False,False,True,False,False,False,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0,81,False,True,False,True,False,False,False,False,True,False,False,True,False,False,True
111,0,73,True,False,False,True,False,False,False,False,True,False,False,True,False,False,True
112,0,56,True,False,False,True,False,False,False,True,False,False,False,True,False,True,False
113,0,74,False,True,False,False,True,False,True,False,False,False,False,True,False,False,True


In [3]:
X=df.drop('Class',axis=1)
y=df['Class']

In [4]:

#split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
random_state=123

In [5]:
#overampling
sm = SMOTE(random_state=random_state)
X_oversampled, y_oversampled = sm.fit_resample(X, y)

X_train_oversampled, X_test_oversampled, y_train_oversampled, y_test_oversampled = train_test_split(X_oversampled, y_oversampled, test_size=0.2, random_state=42)


In [6]:
#underampling
rus = RandomUnderSampler(random_state=random_state)
X_undersampled, y_undersampled = rus.fit_resample(X, y)
X_train_undersampled, X_test_undersampled, y_train_undersampled, y_test_undersampled = train_test_split(X_undersampled, y_undersampled, test_size=0.2, random_state=42)

In [7]:
n_splits = 5

def GridSer_model(model_params,X_train,y_train):
    scores = []
    cv=KFold(n_splits=n_splits, random_state=random_state, shuffle=True)
    for model_name, mp in model_params.items():
        clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False,n_jobs=-1,error_score=0)
        clf.fit(X_train,y_train)
        best_model = mp['model'].set_params(**clf.best_params_)
        scores.append({
            'best_model': best_model,
            'model': model_name,
            'best_score': clf.best_score_,
            'best_params': clf.best_params_
        })
    return pd.DataFrame(scores,columns=['best_model','model','best_score','best_params']).sort_values(by=['best_score'], ascending=False)

In [25]:

def model_training(X_train, y_train, X_test,  y_test, models):
    
    # Create an empty DataFrame to store results
    results_df = pd.DataFrame(columns=['Model', 'Accuracy'])

    for model in models:
        
        model.fit(X_train, y_train)
        # Make predictions
        predictions = model.predict(X_test)
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, predictions)
        precision =precision_score(y_test, predictions)
        recall= recall_score(y_test, predictions)
        cm=confusion_matrix(y_test, predictions)
        
        # Append results to the DataFrame
        temp_df = pd.DataFrame({'Model': [model.__class__.__name__], 'Accuracy': [accuracy], 'Precision': [precision], 'recall': [recall], 'cm':[cm]})
        results_df = pd.concat([results_df, temp_df], ignore_index=True)


    # Sort and return the results DataFrame
    return results_df.sort_values(by=['Accuracy'], ascending=False)


In [9]:
model_params = {
    
    'AdaBoostClassifier' : {
        'model': AdaBoostClassifier(),
        'params': {
            'n_estimators': [50, 100, 200], 
            'learning_rate': [0.01, 0.1, 1]
        }
    } ,
    
    'ExtraTreesClassifier' : {
        'model': ExtraTreesClassifier(),
        'params': {
        'n_estimators': [50, 100, 200],
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 4, 6],
        'min_samples_leaf': [1, 2, 3],
        'max_features': ['auto', 'sqrt', 'log2']
        }
    },
    # 'LGBMClassifier' : {
    #     'model': LGBMClassifier(),
    #     'params': {
    #         'boosting_type': ['gbdt', 'dart', 'goss', 'rf'],
    #         'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
    #         'n_estimators': [50, 100, 200],
    #         'num_leaves': [15, 31, 63, 127],
    #         'min_data_in_leaf': [5, 10, 20],
    #         'lambda_l2': [0.0, 0.1, 1.0]
    #     }
    # },
    'SVM':{
        'model': SVC(),
        'params': {
            'C': [0.1, 1, 10, 100], 
            'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
        }
    },

      'XGBClassifier' : {
        'model': XGBClassifier(),
        'params': {
            'n_estimators': [50, 100, 200], 
            'learning_rate': [0.01, 0.1, 1],
            'max_depth': [3, 5, 10],
            'gamma': [0, 0.1, 0.5, 1],
            'subsample': [0.5, 0.75, 1],
            'colsample_bytree': [0.5, 0.75, 1],
            'lambda': [0.0, 0.1, 1.0]
        }
    },
    'RandomForestClassifier' : {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 4, 6],
            'min_samples_leaf': [1, 2, 3],
            'max_features': ['auto', 'sqrt', 'log2']
        }
    },
    'HistGradientBoostingClassifier':{
        'model': HistGradientBoostingClassifier(),
        'params': {
            'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
            'max_iter': [50, 100, 200],
            'max_leaf_nodes': [15, 31, 63, 127],
            'min_samples_leaf': [5, 10, 20],
            'l2_regularization': [0.0, 0.1, 1.0]
        }
    },
    'LogisticRegression' : {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10, 100], 
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
        }
    }
    
}

In [10]:
original = GridSer_model(model_params,X_train,y_train)

/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it 

In [11]:
original

,best_model,model,best_score,best_params
3,"XGBClassifier(base_score=None, booster=None, c...",XGBClassifier,0.900585,"{'colsample_bytree': 0.5, 'gamma': 0.5, 'lambd..."
5,HistGradientBoostingClassifier(learning_rate=0...,HistGradientBoostingClassifier,0.890643,"{'l2_regularization': 0.0, 'learning_rate': 0...."
2,"SVC(C=1, kernel='linear')",SVM,0.890058,"{'C': 1, 'kernel': 'linear'}"
4,"RandomForestClassifier(max_depth=30, max_featu...",RandomForestClassifier,0.890058,"{'criterion': 'gini', 'max_depth': 30, 'max_fe..."
0,AdaBoostClassifier(learning_rate=0.01),AdaBoostClassifier,0.879532,"{'learning_rate': 0.01, 'n_estimators': 50}"
6,"LogisticRegression(C=1, solver='newton-cg')",LogisticRegression,0.856725,"{'C': 1, 'solver': 'newton-cg'}"
1,"ExtraTreesClassifier(min_samples_leaf=3, min_s...",ExtraTreesClassifier,0.814620,"{'criterion': 'gini', 'max_depth': None, 'max_..."


In [27]:
models=original['best_model'].tolist()
model_training(X_train, y_train, X_test, y_test, models)

/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


,Model,Accuracy,Precision,recall,cm
4,AdaBoostClassifier,0.869565,1.000000,0.7,"[[13, 0], [3, 7]]"
0,XGBClassifier,0.826087,0.875000,0.7,"[[12, 1], [3, 7]]"
2,SVC,0.826087,0.800000,0.8,"[[11, 2], [2, 8]]"
3,RandomForestClassifier,0.826087,0.800000,0.8,"[[11, 2], [2, 8]]"
1,HistGradientBoostingClassifier,0.782609,0.727273,0.8,"[[10, 3], [2, 8]]"
5,LogisticRegression,0.782609,0.777778,0.7,"[[11, 2], [3, 7]]"
6,ExtraTreesClassifier,0.695652,0.636364,0.7,"[[9, 4], [3, 7]]"


In [22]:
oversampled_grd=GridSer_model(model_params,X_train_oversampled,y_train_oversampled)

/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it 

In [23]:
oversampled_grd

,best_model,model,best_score,best_params
3,"XGBClassifier(base_score=None, booster=None, c...",XGBClassifier,0.885238,"{'colsample_bytree': 0.5, 'gamma': 0.1, 'lambd..."
0,"(DecisionTreeClassifier(max_depth=1, random_st...",AdaBoostClassifier,0.884286,"{'learning_rate': 1, 'n_estimators': 100}"
4,"(DecisionTreeClassifier(max_depth=30, max_feat...",RandomForestClassifier,0.864762,"{'criterion': 'entropy', 'max_depth': None, 'm..."
2,"SVC(C=0.1, kernel='linear')",SVM,0.855238,"{'C': 0.1, 'kernel': 'linear'}"
5,HistGradientBoostingClassifier(learning_rate=0...,HistGradientBoostingClassifier,0.854762,"{'l2_regularization': 0.0, 'learning_rate': 0...."
6,"LogisticRegression(C=0.1, solver='newton-cg')",LogisticRegression,0.835714,"{'C': 0.1, 'solver': 'newton-cg'}"
1,"(ExtraTreeClassifier(min_samples_leaf=3, min_s...",ExtraTreesClassifier,0.808095,"{'criterion': 'gini', 'max_depth': 10, 'max_fe..."


In [26]:
models_oversampled_grd=oversampled_grd['best_model'].tolist()
model_training(X_train_oversampled, y_train_oversampled, X_test_oversampled, y_test_oversampled, models_oversampled_grd)

/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


,Model,Accuracy,Precision,recall,cm
3,SVC,0.923077,1.000000,0.857143,"[[12, 0], [2, 12]]"
4,HistGradientBoostingClassifier,0.884615,0.923077,0.857143,"[[11, 1], [2, 12]]"
5,LogisticRegression,0.884615,1.000000,0.785714,"[[12, 0], [3, 11]]"
2,RandomForestClassifier,0.846154,0.857143,0.857143,"[[10, 2], [2, 12]]"
0,XGBClassifier,0.769231,0.785714,0.785714,"[[9, 3], [3, 11]]"
1,AdaBoostClassifier,0.769231,0.785714,0.785714,"[[9, 3], [3, 11]]"
6,ExtraTreesClassifier,0.730769,0.818182,0.642857,"[[10, 2], [5, 9]]"


In [28]:
undersampled_grd=GridSer_model(model_params,X_train_undersampled,y_train_undersampled)

/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it 

In [29]:
undersampled_grd

,best_model,model,best_score,best_params
3,"XGBClassifier(base_score=None, booster=None, c...",XGBClassifier,0.8750,"{'colsample_bytree': 0.5, 'gamma': 0.1, 'lambd..."
4,"(DecisionTreeClassifier(criterion='entropy', m...",RandomForestClassifier,0.8750,"{'criterion': 'entropy', 'max_depth': 30, 'max..."
5,HistGradientBoostingClassifier(learning_rate=0...,HistGradientBoostingClassifier,0.8625,"{'l2_regularization': 0.0, 'learning_rate': 0...."
0,"(DecisionTreeClassifier(max_depth=1, random_st...",AdaBoostClassifier,0.8375,"{'learning_rate': 0.01, 'n_estimators': 50}"
1,"(ExtraTreeClassifier(max_depth=10, min_samples...",ExtraTreesClassifier,0.8375,"{'criterion': 'gini', 'max_depth': 20, 'max_fe..."
2,"SVC(C=1, kernel='linear')",SVM,0.8375,"{'C': 1, 'kernel': 'linear'}"
6,"LogisticRegression(C=10, solver='newton-cg')",LogisticRegression,0.8375,"{'C': 10, 'solver': 'newton-cg'}"


In [30]:
models_undersampled_grd=oversampled_grd['best_model'].tolist()
model_training(X_train_undersampled, y_train_undersampled, X_test_undersampled, y_test_undersampled, models_undersampled_grd)

/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


,Model,Accuracy,Precision,recall,cm
1,AdaBoostClassifier,0.95,0.888889,1.000,"[[11, 1], [0, 8]]"
0,XGBClassifier,0.90,0.800000,1.000,"[[10, 2], [0, 8]]"
2,RandomForestClassifier,0.90,0.800000,1.000,"[[10, 2], [0, 8]]"
3,SVC,0.90,0.800000,1.000,"[[10, 2], [0, 8]]"
4,HistGradientBoostingClassifier,0.90,0.800000,1.000,"[[10, 2], [0, 8]]"
5,LogisticRegression,0.85,0.727273,1.000,"[[9, 3], [0, 8]]"
6,ExtraTreesClassifier,0.85,0.777778,0.875,"[[10, 2], [1, 7]]"
